# Bigram Language Model (from scratch)

In this notebook, Build and train a tiny character-level bigram language model from [Andrej Karpathy’s "Let’s build GPT from scratch"](https://www.youtube.com/watch?v=kCc8FmEb1nY). Keep things simple and self-contained: load Tiny Shakespeare, tokenize at the character level, train a bigram model (each token predicts the next), and sample text.

- It is best to follow this notebook with the Andrej's video



In [ ]:
# Download Tiny Shakespeare once (toy dataset)
# Shakespeare - a concatenation of all of Shakespeare's works in a single file.

!mkdir -p data && \
    wget -O data/input.txt \
    https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-09-05 22:45:18--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘data/input.txt’

data/input.txt      100%[===================>]   1.06M  3.76MB/s    in 0.3s    

2025-09-05 22:45:19 (3.76 MB/s) - ‘data/input.txt’ saved [1115394/1115394]



## Load Text
Read the entire corpus into memory. It’s small (about 1.1M chars), which is perfect for a quick toy experiment.

In [3]:
# Read the dataset
with open("data/input.txt", "r", encoding="utf-8") as f:
    text = f.read()
print(f"Text length: {len(text)}")
print(text[:500])

Text length: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


## Vocabulary and Tokenization
Use a simple character-level tokenizer. That means each unique character becomes a token id. It’s crude compared to BPE/sentencepiece, but perfect for understanding the mechanics.

In [5]:
# Build the character vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print("vocab_size =", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size = 65


In [7]:
# Character-level tokenizer: char <-> id
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # string -> list[int]
decode = lambda l: "".join([itos[i] for i in l])  # list[int] -> string

print(encode("Hello, there!"))
print(decode(encode("Hello, there!")))

[20, 43, 50, 50, 53, 6, 1, 58, 46, 43, 56, 43, 2]
Hello, there!


## Numericalize and Split
Convert the full corpus to token ids (a long 1D tensor), then keep 90% for training and 10% for validation.

In [8]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# Train/val split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

## Context Windows
- It would be computationally very expensive and prohibitive to feed entire text into a Transformer all at once. When we actually train a Transformer on large datasets, we only work with chunks of the dataset. During training, we sample random chunks from the training set and train on just these chunks at a time. These chunks have a maximum length called `block_size`.

- When we plug data into a Transformer, we simultaneously train it to make predictions at every position. In a chunk of nine characters, there are actually eight individual examples packed in. 
- We train on all of these not just for computational reasons or efficiency, but also to make the Transformer network accustomed to seeing contexts ranging from as little as one character all the way up to block_size. 
- This is useful later during inference because while sampling, we can start generation with as little as one character of context, and the Transformer knows how to predict the next character with contexts ranging from one all the way up to block_size. 
- After block_size, we have to start truncating because the Transformer will never receive more than block_size inputs when predicting the next character.

In [10]:
block_size = 8  # maximum context length
x = train_data[:block_size]
y = train_data[1 : block_size + 1]
for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"Example {t+1} -> input {context.tolist()} -> target {int(target)}")

Example 1 -> input [18] -> target 47
Example 2 -> input [18, 47] -> target 56
Example 3 -> input [18, 47, 56] -> target 57
Example 4 -> input [18, 47, 56, 57] -> target 58
Example 5 -> input [18, 47, 56, 57, 58] -> target 1
Example 6 -> input [18, 47, 56, 57, 58, 1] -> target 15
Example 7 -> input [18, 47, 56, 57, 58, 1, 15] -> target 47
Example 8 -> input [18, 47, 56, 57, 58, 1, 15, 47] -> target 58


Note, during inference after reaching the block_size, we have to start truncating because the Transformer will never receive more than the `block_size` inputs when it's predicting the next character

## Mini-batching
Sample random starting positions to build batches of shape `(batch_size, block_size)`. Each row is an independent training example. Batching keeps the GPU/CPU busy and speeds up training.
> In Transformers, we have many batches of multiple chunks of text that are stacked up in a single tensor. This is done for efficiency to keep the GPUs busy, as they are very good at parallel processing of data.

In [11]:
torch.manual_seed(42)
batch_size = 4  # number of sequences processed in parallel


def get_batch(split):
    source = train_data if split == "train" else val_data
    ix = torch.randint(len(source) - block_size, (batch_size,))
    x = torch.stack([source[i : i + block_size] for i in ix])
    y = torch.stack([source[i + 1 : i + 1 + block_size] for i in ix])
    return x, y


xb, yb = get_batch("train")
print("xb", xb.shape)
print("yb", yb.shape)
# Peek a few (context, target) pairs
for xrow, yrow in zip(xb, yb):
    for t in range(block_size):
        print(f"input {xrow[:t+1].tolist()} -> target {int(yrow[t])}")
    print("---")

xb torch.Size([4, 8])
yb torch.Size([4, 8])
input [57] -> target 1
input [57, 1] -> target 46
input [57, 1, 46] -> target 47
input [57, 1, 46, 47] -> target 57
input [57, 1, 46, 47, 57] -> target 1
input [57, 1, 46, 47, 57, 1] -> target 50
input [57, 1, 46, 47, 57, 1, 50] -> target 53
input [57, 1, 46, 47, 57, 1, 50, 53] -> target 60
---
input [1] -> target 58
input [1, 58] -> target 46
input [1, 58, 46] -> target 43
input [1, 58, 46, 43] -> target 56
input [1, 58, 46, 43, 56] -> target 43
input [1, 58, 46, 43, 56, 43] -> target 1
input [1, 58, 46, 43, 56, 43, 1] -> target 41
input [1, 58, 46, 43, 56, 43, 1, 41] -> target 39
---
input [17] -> target 26
input [17, 26] -> target 15
input [17, 26, 15] -> target 17
input [17, 26, 15, 17] -> target 10
input [17, 26, 15, 17, 10] -> target 0
input [17, 26, 15, 17, 10, 0] -> target 32
input [17, 26, 15, 17, 10, 0, 32] -> target 53
input [17, 26, 15, 17, 10, 0, 32, 53] -> target 1
---
input [57] -> target 58
input [57, 58] -> target 6
input [57

## Bigram Model
- In language modeling, the Bigram language model is probably the simplest language model
- It use a single embedding table of shape `(vocab_size, vocab_size)`. Given a token id, look up a row and interpret it directly as the logits for the next token. 
    > Embedding Table: This is basically a tensor of shape vocab_size by vocab_size. When we pass indices here, every single integer in our input refers to this embedding table and plucks out a row from that embedding table. We interpret this as the logits, which are the scores for the next character in the sequence.
    > What's happening is we're predicting what comes next based solely on the individual identity of a single token. 
- There’s no context mixing here — it’s a pure bigram model.
- Training is done with `cross-entropy loss`.
    > Cross entropy loss is negative log likelihood loss. The loss is the cross entropy between the predictions and the targets, which measures the quality of the logits with respect to the targets.  


In [12]:
import torch.nn as nn
from torch.nn import functional as F
from einops import rearrange


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size: int):
        super().__init__()
        # Each token id maps to a row of logits for the next token
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, inputs, targets=None):
        # inputs: (B, T) -> logits: (B, T, C) with C=vocab_size
        logits = self.token_embedding_table(inputs)
        loss = None
        if targets is not None:
            # Flatten batch and time for cross-entropy
            logits = rearrange(logits, "b t c -> (b t) c")
            targets = rearrange(targets, "b t -> (b t)")
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, inputs, max_new_tokens: int):
        # Autoregressively sample `max_new_tokens` tokens
        for _ in range(max_new_tokens):
            logits, _ = self(inputs)  # (B, T, C)
            logits = logits[:, -1, :]  # only last step: (B, C)
            probs = F.softmax(logits, dim=-1)  # convert to probabilities
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            inputs = torch.cat((inputs, idx_next), dim=1)  # append
        return inputs


m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print("logits shape:", out.shape, "| loss:", loss.item())

logits shape: torch.Size([32, 65]) | loss: 4.724130630493164


We expect the loss to be about -ln(1/65), which is approximately 4.17, but we're getting 4.72. This tells us that the initial predictions are not super diffuse - they have a little bit of structure already.

In [13]:
# Untrained sample (will be gibberish, but shows the flow)
start = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(start, max_new_tokens=100)[0].tolist()))


cfYCDRUZsYBsA?Y?vgB!ZWOEiAoezL:q&Avufr?gSGdWrp&Bxt-R?wo'TYhBChdIC-RDaRmEGENyouVg'UjyQNyQSpZUVeN:BZqh


## Train
Use `AdamW` optimizer. A typical good setting for the learning rate is roughly `3e-4`. However, for tiny models we can use a relatively large learning rate (`1e-3`), 

In [14]:
batch_size = 32
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
for step in range(10000):
    xb, yb = get_batch("train")
    _, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if step % 200 == 0:
        print(f"step {step:5d} | loss {loss.item():.4f}")

step     0 | loss 4.7358
step   200 | loss 4.5046
step   400 | loss 4.2356
step   600 | loss 4.0194
step   800 | loss 3.8142
step  1000 | loss 3.6808
step  1200 | loss 3.5124
step  1400 | loss 3.4114
step  1600 | loss 3.3483
step  1800 | loss 3.1592
step  2000 | loss 3.1517
step  2200 | loss 2.9603
step  2400 | loss 3.0527
step  2600 | loss 2.9030
step  2800 | loss 2.7741
step  3000 | loss 2.7933
step  3200 | loss 2.7562
step  3400 | loss 2.6762
step  3600 | loss 2.7054
step  3800 | loss 2.7220
step  4000 | loss 2.6779
step  4200 | loss 2.5870
step  4400 | loss 2.5370
step  4600 | loss 2.5282
step  4800 | loss 2.6146
step  5000 | loss 2.5610
step  5200 | loss 2.5641
step  5400 | loss 2.6263
step  5600 | loss 2.5751
step  5800 | loss 2.5393
step  6000 | loss 2.4923
step  6200 | loss 2.6814
step  6400 | loss 2.4553
step  6600 | loss 2.4046
step  6800 | loss 2.5248
step  7000 | loss 2.5210
step  7200 | loss 2.4300
step  7400 | loss 2.2994
step  7600 | loss 2.5724
step  7800 | loss 2.5194


## Sample
Finally, generate text by sampling one token at a time from the model’s predicted distribution

In [16]:
start = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(start, max_new_tokens=300)[0].tolist()))


ORThay garomy CHA hthe IZ:
Then'swilisteat
HERI w bakee ckinthy
S:
CIf ditomaisonute ghilyow bloker twiartofre.
HE:
nse,
CII'l t Sthenearoor t ickorathisithet,
ICEvel hilichos thton.
AMat y thiseefos I acqust mee sprane id
TAMisheedeatof ss,
Mowiler anes th ct f aiorthranins f tur;
tes thy, cit.
Sts
